In [158]:
import pickle
import numpy as np
from copy import deepcopy
import import_ipynb
from greedy_algo import *
import matplotlib.pyplot as plt


In [159]:
def Ber(p):
    #Gives a random sample from Bernoulli distribution with prob. of 1 as p
    return np.random.binomial(n=1,p=p,size=1).item()


In [160]:
#Base Graph G=(V,E)
V = list(range(20))
E = {}

for v in V:
    E[v] = []
    
for v in V:
    for vv in V:
        if(v != vv and v > vv):
            if(np.random.rand(1)>0.5):
                E[v].append(vv)
                E[vv].append(v)

G = [V,E]
print(E)
#This piece of code generates true edge probabilities we have for G=(V,E)
p = np.zeros((len(V),len(V)))
degree = []
for v in V:
    degree.append(len(E[v]))

for v in V:
    for e in E[v]:
        p[v][e] = 1/degree[e]

{0: [1, 3, 7, 10, 13, 14, 17], 1: [0, 7, 10, 11, 13, 18], 2: [3, 4, 5, 7, 8, 12, 13, 17, 18], 3: [0, 2, 7, 9, 12, 14], 4: [2, 7, 8, 11, 12, 14, 15, 16, 18, 19], 5: [2, 6, 7, 8, 9, 12, 13, 14, 15, 16, 18, 19], 6: [5, 7, 9, 12, 15, 18], 7: [0, 1, 2, 3, 4, 5, 6, 8, 10, 17, 18], 8: [2, 4, 5, 7, 10, 14, 16, 17, 18, 19], 9: [3, 5, 6, 12, 13, 14, 15, 16], 10: [0, 1, 7, 8, 12, 13, 16, 17, 18, 19], 11: [1, 4, 14, 15, 16, 17, 18, 19], 12: [2, 3, 4, 5, 6, 9, 10, 13, 14, 17, 18, 19], 13: [0, 1, 2, 5, 9, 10, 12, 14, 15, 16, 17, 18, 19], 14: [0, 3, 4, 5, 8, 9, 11, 12, 13, 18, 19], 15: [4, 5, 6, 9, 11, 13, 17, 18, 19], 16: [4, 5, 8, 9, 10, 11, 13, 18], 17: [0, 2, 7, 8, 10, 11, 12, 13, 15, 18, 19], 18: [1, 2, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14, 15, 16, 17], 19: [4, 5, 8, 10, 11, 12, 13, 14, 15, 17]}


In [161]:
#Gerenrating a sub-graph of G (say Gt=(Vt,Et))

def give_Gt(G,p):
    #G is base graph
    #p is prob. that an edge in G is present
    (V,E) = G
    Vt = deepcopy(V)
    Et = {}
    for v in V:
        Et[v]=[]

    for v in V:
        for e in E[v]:
            if(e>v and Ber(p)==1):
                Et[v].append(e)
                Et[e].append(v)
    return (Vt,Et)

In [162]:
#Settings
T = int(1e+3)
k = 2#we need 2 most influenced nodes
no_of_vertices = len(V)
#p we have already
p_sum = np.zeros((no_of_vertices,no_of_vertices))
N_t = np.zeros((no_of_vertices,no_of_vertices))


In [163]:
def any_unpulledarm(Vt,Et):
    for v in Vt:
        for e in Et[v]:
            if(N_t[v][e]==0):
                return 1
    return 0

In [164]:
regret_t = []
regret_sum = 0

for t in range(1,T+1):
    Gt = give_Gt(G,p=1)
    (Vt,Et) = Gt
    
    if(any_unpulledarm(Vt,Et)):
        #pull all arms Et
        for v in Vt:
            for e in Et[v]:
                sample = Ber(p[v][e])
                p_sum[v][e] += sample
                N_t[v][e] += 1
        regret_t.append(0)

        #print(p_sum,N_t)          
    else:
        #ORACLE
        #print(p_bar-p)
        gamma = (1-(1/np.exp(1)))
        beta = 1
        #p_bar = p_sum/N_t + np.sqrt((3*np.log10(t))/(20*N_t[v][e]))
        (St,_,_) = greedy(Gt,k,p_sum,N_t,t,mc=100)
        (r_St,pulled_arms) = IC2(Gt,St,p,mc=1)
        for v in list(pulled_arms.keys()):
            for e in pulled_arms[v]:
                p_sum[v][e] += 1
                N_t[v][e] += 1
        # for v in Vt:
        #     for e in Et[v]:        
        #         N_t[v][e] += 1
        #         temp = (p_sum[v][e]/N_t[v][e]) + np.sqrt((3*np.log10(t))/(2*N_t[v][e])) 
        #         if(temp>1):
        #             p_bar[v][e] = 1
        #         else:
        #             p_bar[v][e] = temp
        (St_star,_,_) = greedy2(Gt,k,p,mc=100)
        r_St_star,_ = IC2(Gt,St_star,p,mc=1)
        regret_sum += 1*beta*r_St_star - r_St
        regret_t.append(regret_sum)
        print(St,St_star,r_St,r_St_star)
        plt.plot(regret_t,color='blue')
        plt.savefig('plot.png')
        plt.close()



[1, 3] [18, 5] 2.0 3.0
[1, 3] [18, 5] 2.0 3.0
[0, 1] [18, 5] 2.0 3.0
[0, 1] [18, 5] 2.0 3.0
[0, 1] [18, 5] 2.0 3.0
[0, 1] [18, 5] 2.0 3.0
